In [1]:
from __future__ import annotations
import pandas as pd
import numpy as np

# --- Load data ---
xlsx_path = "Test Data.xlsx"  # change path if needed
sales  = pd.read_excel(xlsx_path, sheet_name="Sales")
agents = pd.read_excel(xlsx_path, sheet_name="Agent Commission in %")
prod   = pd.read_excel(xlsx_path, sheet_name="Product Info")

# --- Parse dates robustly (no fragile string formats) ---
sales["Transaction Date"]   = pd.to_datetime(sales["Transaction Date"], errors="coerce")
prod["From Date"]           = pd.to_datetime(prod["From Date"], errors="coerce")
prod["To Date"]             = pd.to_datetime(prod["To Date"], errors="coerce")
agents["Valid Date From"]   = pd.to_datetime(agents["Valid Date From"], errors="coerce")

# Open-ended product ranges: set To Date to far future so they match any later TX date
FAR_FUTURE = pd.Timestamp("2262-04-11")
prod["To Date"] = prod["To Date"].fillna(FAR_FUTURE)

# Drop rows that can’t participate in joins
sales = sales.dropna(subset=["Product Code", "Transaction Date"]).copy()
prod  = prod.dropna(subset=["Product Code", "From Date"]).copy()

# --- Map Product Category to each sale by Product Code & date window ---
# We use a per-product merge_asof to avoid strict global sort requirements.
pieces = []
prod_codes = sorted(set(sales["Product Code"]).intersection(set(prod["Product Code"])))

for code in prod_codes:
    s = sales.loc[sales["Product Code"] == code].sort_values("Transaction Date").copy()
    p = prod.loc[prod["Product Code"] == code].sort_values("From Date").copy()
    if len(s) == 0 or len(p) == 0:
        continue
    m = pd.merge_asof(
        s, p,
        left_on="Transaction Date",
        right_on="From Date",
        direction="backward",
        allow_exact_matches=True
    )
    # keep only rows where TX date is within [From Date, To Date]
    m = m[m["Transaction Date"] <= m["To Date"]]
    pieces.append(m)

merged = pd.concat(pieces, ignore_index=True)

# --- Melt agent commissions to long and keep units as PERCENT (not fraction) ---
agents_long = agents.melt(
    id_vars=["Agent Code", "Valid Date From"],
    var_name="Product Category",
    value_name="Commission"
)

def to_percent(x):
    """Return commission as a float PERCENT (e.g., 5, 7, 15)."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s.endswith("%"):
        try:
            return float(s[:-1])
        except:
            return np.nan
    try:
        return float(s)  # assume already in percent units
    except:
        return np.nan

agents_long["Commission"] = agents_long["Commission"].apply(to_percent)

# --- For each (Agent, Category), pick most recent commission as of TX date ---
merged = merged.dropna(subset=["Agent Code", "Product Category", "Transaction Date"]).copy()

agent_pieces = []
keys = merged[["Agent Code","Product Category"]].drop_duplicates().itertuples(index=False, name=None)

for ag, cat in keys:
    left  = merged[(merged["Agent Code"]==ag) & (merged["Product Category"]==cat)].sort_values("Transaction Date").copy()
    right = agents_long[(agents_long["Agent Code"]==ag) & (agents_long["Product Category"]==cat)].sort_values("Valid Date From").copy()
    if len(left)==0:
        continue
    if len(right)==0:
        left["Commission"] = np.nan
        agent_pieces.append(left)
        continue
    mm = pd.merge_asof(
        left, right,
        left_on="Transaction Date",
        right_on="Valid Date From",
        direction="backward",
        allow_exact_matches=True
    )
    agent_pieces.append(mm)

merged2 = pd.concat(agent_pieces, ignore_index=True)
merged2["Commission"] = merged2["Commission"].fillna(0.0)

# --- Normalize any merge suffixes so groupbys use your original names ---
df = merged2.copy()
if "Agent Code" not in df.columns:
    if "Agent Code_x" in df.columns: df.rename(columns={"Agent Code_x":"Agent Code"}, inplace=True)
    elif "Agent Code_y" in df.columns: df.rename(columns={"Agent Code_y":"Agent Code"}, inplace=True)
if "Product Category" not in df.columns:
    if "Product Category_x" in df.columns: df.rename(columns={"Product Category_x":"Product Category"}, inplace=True)
    elif "Product Category_y" in df.columns: df.rename(columns={"Product Category_y":"Product Category"}, inplace=True)

# --- Reproduce your six outputs (Commission remains PERCENT) ---
print("Commission per Agent")
print(df.groupby("Agent Code")["Commission"].mean()); print()

print("Commission per Product Category")
print(df.groupby("Product Category")["Commission"].mean()); print()

print("Number of Sales per Agent")
print(df.groupby("Agent Code")["Agent Code"].count()); print()

print("Number of Sales per Product")
# Matches your original (counts by Product Category)
print(df.groupby("Product Category")["Product Category"].count()); print()

print("A matrix of the number of sales per agent per product category")
print(df.groupby(["Agent Code","Product Category"])["Transaction ID"].count()); print()

print("A matrix of the commission value sales per agent per product category.")
# As in your code, this is the mean commission RATE (%), not a money value
print(df.groupby(["Agent Code","Product Category"])["Commission"].mean())


Commission per Agent
Agent Code
A001    5.392281
A002    7.242443
A003    6.648550
A004    9.942113
Name: Commission, dtype: float64

Commission per Product Category
Product Category
Drinks         8.663092
Frozen Food    9.967784
Fruit          6.754782
Other Food     5.780321
Name: Commission, dtype: float64

Number of Sales per Agent
Agent Code
A001    12644
A002    12902
A003    12898
A004    12870
Name: Agent Code, dtype: int64

Number of Sales per Product
Product Category
Drinks         17622
Frozen Food     3042
Fruit          15578
Other Food     15072
Name: Product Category, dtype: int64

A matrix of the number of sales per agent per product category
Agent Code  Product Category
A001        Drinks              4380
            Frozen Food          732
            Fruit               3869
            Other Food          3663
A002        Drinks              4344
            Frozen Food          775
            Fruit               3977
            Other Food          3806
A003   